In [24]:
import torch
import torch.nn.functional as F
import numpy as np
from PIL import Image



In [25]:
# ---- load and prep image (grayscale, [B,C,H,W]) ----
img_path = "1657999067686 copy.jpeg"  # or the full path to your image
img = Image.open(img_path).convert("RGB")  # grayscale


In [26]:
x = torch.from_numpy(np.array(img)).float() / 255.0  # [H,W] in [0,1]
x = x.unsqueeze(0) # [1,1,H,W]

# ---- define 3x3 Sobel kernels ----
kx = torch.tensor([[-2., 0., 1.],
                   [-2., 0., 2.],
                   [-1., 0., 1.]]).reshape(1,1,3,3)

ky = torch.tensor([[-1., -2., -1.],
                   [ 0.,  0.,  0.],
                   [ 1.,  2.,  1.]]).reshape(1,1,3,3)



In [27]:
# move kernels to same device as input
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = x.to(device)
kx = kx.to(device)
ky = ky.to(device)



In [28]:
# ---- convolution with padding=1 to keep size ----
gx = F.conv2d(x, kx, padding=1)
gy = F.conv2d(x, ky, padding=1)

# ---- gradient magnitude ----
mag = gx+gy



RuntimeError: Given groups=1, weight of size [1, 1, 3, 3], expected input[1, 800, 800, 3] to have 1 channels, but got 800 channels instead

In [19]:
# ---- normalize to [0,255] and save ----
m = mag.squeeze().detach().cpu().numpy()
m = (m / (m.max() + 1e-8) * 255.0).astype(np.uint8)
out = Image.fromarray(m)
out.save("edges_sobel.jpg")
print("Saved edges to edges_sobel.jpg")

Saved edges to edges_sobel.jpg


In [13]:
laplace_k = torch.tensor([[0.,  1., 0.],
                          [1., -4., 1.],
                          [0.,  1., 0.]]).reshape(1,1,3,3).to(device)
lap = F.conv2d(x, laplace_k, padding=1)
m = lap.squeeze().abs().detach().cpu().numpy()
m = (m / (m.max() + 1e-8) * 255.0).astype(np.uint8)
Image.fromarray(m).save("edges_laplacian.jpg")

In [33]:
import torch
import torch.nn.functional as F
import numpy as np
from PIL import Image

# ---------- load RGB image -> [1,3,H,W] float in [0,1] ----------
img_path = "1657999067686 copy.jpeg"  # or the full path to your image
img = Image.open(img_path).convert("RGB")
x = torch.from_numpy(np.array(img)).float() / 255.0            # [H,W,3]
x = x.permute(2, 0, 1).unsqueeze(0)                            # [1,3,H,W]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = x.to(device)

# ---------- choose one 3×3 smoothing kernel (sums to 1.0) ----------
# Box blur (mean filter)
box = torch.tensor([[1., 1., 1.],
                    [1., 1., 1.],
                    [1., 1., 1.]]) / 9.0

# Gaussian blur (sigma≈1)
gauss = torch.tensor([[1., 2., 1.],
                      [2., 4., 2.],
                      [1., 2., 1.]]) / 16.0

K = gauss  # <- switch to `box` if you want mean blur

# Repeat kernel per channel and move to device
K3 = K.view(1, 1, 3, 3).repeat(3, 1, 3, 3).to(device)  # [out=3, in_per_group=1, 3,3]

# ---------- reflect-pad to avoid dark borders, then grouped conv ----------
# (conv2d doesn't support reflect directly; we pad first then use padding=0)
x_pad = F.pad(x, pad=(1, 1, 1, 1), mode="reflect")  # left, right, top, bottom
y = F.conv2d(x_pad, K3, padding=0, groups=3)        # [1,3,H,W]

# ---------- save result ----------
y_np = (y.squeeze(0).clamp(0, 1) * 255.0).byte().permute(1, 2, 0).cpu().numpy()
Image.fromarray(y_np).save("smoothed_3x3.jpg")
print("Saved smoothed image to smoothed_3x3.jpg")

Saved smoothed image to smoothed_3x3.jpg
